# Visualización de vectores de palabras con t-SNE

TSNE es una técnica útil a aplicar cuando se trata de visualizar la similitud entre objetos. Funciona tomando un grupo de vectores de características de palabras de vocabulario de alta dimensión (100 dimensiones a través de Word2Vec), y luego los comprime a pares de coordenadas x,y de 2 dimensiones. La idea es mantener las palabras similares cerca en el plano, mientras se maximiza la distancia entre las palabras no similares.

Pasos
* Limpiar los datos
* Construir un corpus
* Entrenar un modelo Word2Vec
* Visualizar las representaciones t-SNE de las palabras más comunes

Original y crédito: https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne y 
https://github.com/rouseguy/DeepLearning-NLP

El corpus a usar es Quora Question Pairs (Pares de preguntas de Quora)

Perteneció a un concurso bajo la pregunta ¿Puedes identificar los pares de preguntas que tienen la misma intención?. El objetivo de este concurso fue predecir cuáles de los pares de preguntas proporcionados contienen dos preguntas con el mismo significado. La verdad básica es el conjunto de etiquetas que han sido suministradas por expertos humanos. Las etiquetas de la verdad básica son intrínsecamente subjetivas, ya que el verdadero significado de las frases nunca puede conocerse con certeza. Además, el etiquetado humano es un proceso "ruidoso" y las personas razonables pueden discrepar. Por lo tanto, las etiquetas de la verdad sobre el terreno en este conjunto de datos deben considerarse "informadas", pero no 100% exactas, y pueden incluir un etiquetado incorrecto. Creemos que las etiquetas, en general, representan un consenso razonable, pero a menudo esto puede no ser cierto en cada caso para los elementos individuales en el conjunto de datos.

Estructura  de datos
* id - el id de un par de preguntas del conjunto de entrenamiento
* qid1, qid2 - identificadores únicos de cada pregunta (sólo disponibles en train.csv)
* question1, question2 - el texto completo de cada pregunta
* is_duplicate - la variable de destino, establecida en 1 si la pregunta1 y la pregunta2 tienen esencialmente el mismo significado, y 0 en caso contrario.


In [ ]:
# Carga de librerías

import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

# Carga del conjunto de datos en un dataframe
data = pd.read_csv('../input/quora-question-pairs/train.csv.zip').sample(50000, random_state=23)

# carga de conjunto de datos de palabra vacías
STOP_WORDS = nltk.corpus.stopwords.words()

# Procedimiento para eliminar los caracteres que no son letras o números, reducir las mayúsculas, 
# y luego eliminar las palabras vacías

def clean_sentence(val): 
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    
    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

# Procedimiento para borrar nans, luego aplicar la función 'clean_sentence' a la pregunta1 y 2"

def clean_dataframe(data):
    data = data.dropna(how="any")
    
    for col in ['question1', 'question2']:
        data[col] = data[col].apply(clean_sentence)
    
    return data

data = clean_dataframe(data)

data.head(5) # mostrar daatos limpios

In [ ]:
def build_corpus(data): #  Crea una lista de listas con palabras de cada pregunta q
    corpus = []
    for col in ['question1', 'question2']:
        for sentence in data[col].iteritems():
            word_list = sentence[1].split(" ")
            corpus.append(word_list)
            
    return corpus

corpus = build_corpus(data)        

print(len(corpus)) # Cantidad de elementos en la lista armada
print(corpus[0:4]) # Se muestran los primeros 4 elementos de la lista


In [ ]:
# la técnica Word2Vec produce un vocabulario en el que cada palabra está representada por una 
# matriz numpy n-dimensional (100 valores en este ejemplo)

model = word2vec.Word2Vec(corpus, vector_size=100, window=20, min_count=200, workers=4)

model.wv['trump']

In [ ]:
# Se aplica la técnica TSNE para visualizar datos y sus relaciones

def tsne_plot(model): # Se crea un modelo tsne y se grafican los datos
    
    labels = []
    tokens = []

    for word in list(model.wv.key_to_index):
        tokens.append(model.wv[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 30)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

tsne_plot(model)

In [ ]:
# Un modelo más selectivo, tiene en cuenta las palabras que al menos tienen 500 repeticiones en el corpus

model = word2vec.Word2Vec(corpus, vector_size=100, window=20, min_count=500, workers=4)

tsne_plot(model)

In [ ]:
# Con un conjunto de datos tan grande, es difícil hacer una visualización de TSNE fácil de leer. Lo que se puede 
# hacer es utilizar el modelo para buscar las palabras más parecidas a partir de un punto determinado.

model.wv.most_similar('trump')

In [ ]:
model.wv.most_similar('university')